In [2]:
"""
Os dados utilizados para a análise foram obtidos do site do INMET (Instituto Nacional de Meteorologia) e estão disponíveis em: https://bdmep.inmet.gov.br/#
Para realizar a análise, é necessário baixar os dados das estações meteorológicas de interesse e salvá-los no mesmo diretório do código.

O objetivo princiapal é analisar a temperatura média das estações meteorológicas de interesse e compará-las com o gráfico de temperatura média do INMET. O valor escolhido
será entrada para a varíavel Reference Air Temeprature (°C) do modelo Urban Cooling do InVEST.

"""
# Bibliotecas utilizadas para a análise dos dados climáticos
import os # Biblioteca para manipulação de arquivos
import pandas as pd # Biblioteca para manipulação de dados
import plotly.graph_objects as go # Biblioteca para plotagem de gráficos

In [22]:
# Caminho do arquivo
def caminho_arquivo(nome_arquivo: str):
    return os.path.join(os.getcwd(), nome_arquivo)

# Carregar dados do arquivo CSV
def carregar_dados_csv(caminho_arquivo: str):
    return pd.read_csv(caminho_arquivo, delimiter=';', skiprows=10)

# Contar valores nodata a partir de uma coluna de interesse
def contar_valores_nodata(df: pd.DataFrame, coluna: str):
    isna = df[coluna].isna()
    notna = df[coluna].notna()
    return f'Completo: {len(df[coluna])} | Vazios: Com vazios: {isna.sum()}  | Sem vazios: {notna.sum()}'

# Filtrar dados não nulos
def filtrar_dados_notNaN(df: pd.DataFrame, coluna: str):
    notna = df[coluna].notna()
    return df[notna]

# Inserir coluna com o número do mês
def inserir_mes_ano(df: pd.DataFrame, coluna: str):
    df_copy = df.copy()
    df_copy[coluna] = pd.to_datetime(df_copy[coluna]) # Converter a coluna para o formato de data
    nm_mes = df_copy[coluna].dt.strftime("%B") # Extrair o nome do mês
    ano = df_copy[coluna].dt.strftime("%Y") # Extrair o ano

    df_copy.insert(1,"MÊS",nm_mes) # Inserir na 1ª posição a coluna com o nome do mês
    df_copy.insert(2,"ANO",ano) # Inserir na 1ª posição a coluna com o ano

    return df_copy

# Inserir coluna com o número do mês e orderar
def inserir_numero_mes(df: pd.DataFrame):
    df_copy = df.copy()

    # Definir um dicionário com o número do mês
    dt_number = {'': 0,
                'January': 1,
                'February': 2,
                'March': 3,
                'April': 4,
                'May': 5,
                'June': 6,
                'July': 7,
                'August': 8,
                'September': 9,
                'October': 10,
                'November': 11,
                'December': 12
                }

    # Relacionar o número do mês com o nome do mês e atribuir a varíavel num_mes
    num_mes = df_copy['MÊS'].apply(lambda x: dt_number[x])

    df_copy.insert(1,"NUM_MÊS",num_mes) # Inserir na 1ª posição a coluna com o número do mês

    df_copy = df_copy.sort_values(by=['NUM_MÊS']) # Ordenar os dados pela coluna NUM_MÊS

    return df_copy

# Anos que possuem menos de 360 dias de dados de temperatura média
def listar_anos_descartados(df: pd.DataFrame, coluna: str):
    df_copy = df.copy()

    qtd_dias_ano = df_copy.groupby([coluna]).count() # Contar a quantidade de dias de medição por ano
    qtd_dias_ano = qtd_dias_ano.reset_index() # Resetar o índice

    qtd_dias_ano = qtd_dias_ano.iloc[:,0:2] # Selecionar as colunas de interesse
    qtd_dias_ano = qtd_dias_ano.rename(columns={"Data Medicao": "Dias"}) # Renomear a coluna

    l_ano_maior = qtd_dias_ano[qtd_dias_ano['Dias'] > 360]['ANO'].tolist()   # Anos com mais de 360 dias de medição
    l_ano_menor = qtd_dias_ano[qtd_dias_ano['Dias'] < 360]['ANO'].tolist() # Anos com menos de 360 dias de medição
    
    return l_ano_maior, l_ano_menor

def df_temperatura_media_mes(df: pd.DataFrame, colunas: list, coluna: str):
    df_copy = df.copy()

    # Média da Temperatural Mensal para cada ano e mês da série temporal
    temp_mes_serie = df_copy.groupby(colunas)[coluna].mean().reset_index()
    return temp_mes_serie

def df_temperatura_media_ano_completo(df: pd.DataFrame, colunas: list, coluna: str):
    df_copy = df.copy()

    # Média da Temperatural Mensal para cada ano e mês da série temporal
    temp_mes = df_copy.groupby(colunas)[coluna].mean()

    df2 = pd.DataFrame(temp_mes)
    df2 = df2.reset_index()
    return df2

In [4]:

# Carregar os dados do arquivo excel a um df 
df_bsb_83377_d = carregar_dados_csv(caminho_arquivo('dados_83377_D_1961-09-11_2024-03-14.csv')) # Estação Convêncional do Inmet Brasília - 83377
df_bsb_A001_d = carregar_dados_csv(caminho_arquivo('dados_A001_D_2000-05-06_2024-03-14.csv')) # Estção Automática do Inmet Brasília - A001
df_AguasEmendadas_A045_d = carregar_dados_csv(caminho_arquivo('dados_A045_D_2008-10-02_2024-03-14.csv')) # Estação Automática do Inmet Águas Emendadas - A045


In [6]:
contar_valores_nodata(df_bsb_83377_d, 'TEMPERATURA MEDIA COMPENSADA, DIARIA(°C)')


'Completo: 22831 | Vazios: Com vazios: 269  | Sem vazios: 22562'

In [7]:
contar_valores_nodata(df_bsb_A001_d,'TEMPERATURA MEDIA, DIARIA (AUT)(°C)' )

'Completo: 8714 | Vazios: Com vazios: 340  | Sem vazios: 8374'

In [8]:
contar_valores_nodata(df_AguasEmendadas_A045_d,'TEMPERATURA MEDIA, DIARIA (AUT)(°C)' )

'Completo: 5643 | Vazios: Com vazios: 137  | Sem vazios: 5506'

In [19]:
df_bsb_83377_d_filtro = filtrar_dados_notNaN(df_bsb_83377_d, 'TEMPERATURA MEDIA COMPENSADA, DIARIA(°C)')
df_bsb_A001_d_filtro = filtrar_dados_notNaN(df_bsb_A001_d, 'TEMPERATURA MEDIA, DIARIA (AUT)(°C)')
df_AguasEmendadas_A045_d_filtro = filtrar_dados_notNaN(df_AguasEmendadas_A045_d, 'TEMPERATURA MEDIA, DIARIA (AUT)(°C)')


In [20]:
df_bsb_83377_d_filtro = inserir_mes_ano(df_bsb_83377_d_filtro, 'Data Medicao')
df_bsb_A001_d_filtro = inserir_mes_ano(df_bsb_A001_d_filtro, 'Data Medicao')
df_AguasEmendadas_A045_d_filtro = inserir_mes_ano(df_AguasEmendadas_A045_d_filtro, 'Data Medicao')

In [23]:
df_bsb_83377_d_filtro = inserir_numero_mes(df_bsb_83377_d_filtro)
df_bsb_A001_d_filtro = inserir_numero_mes(df_bsb_A001_d_filtro)
df_AguasEmendadas_A045_d_filtro = inserir_numero_mes(df_AguasEmendadas_A045_d_filtro)

In [24]:
est_bsb_conv_temp = df_bsb_83377_d_filtro[['Data Medicao','NUM_MÊS','MÊS','ANO','TEMPERATURA MEDIA COMPENSADA, DIARIA(°C)']]
est_bsb_aut_temp = df_bsb_A001_d_filtro[['Data Medicao','NUM_MÊS','MÊS','ANO','TEMPERATURA MEDIA, DIARIA (AUT)(°C)']]
est_emendadas_aut_temp = df_AguasEmendadas_A045_d_filtro[['Data Medicao','NUM_MÊS','MÊS','ANO','TEMPERATURA MEDIA, DIARIA (AUT)(°C)']]


In [25]:
completos_bsb_conv, incompletos_bsb_conv = listar_anos_descartados(est_bsb_conv_temp, 'ANO')
print(f'{len(completos_bsb_conv)}, {len(incompletos_bsb_conv)}')


57, 6


In [26]:
completos_bsb_aut, incompletos_bsb_aut = listar_anos_descartados(est_bsb_aut_temp, 'ANO')
print(f'{len(completos_bsb_aut)}, {len(incompletos_bsb_aut)}')

13, 11


In [27]:
completos_emendadas_aut, incompletos_emendadas_aut = listar_anos_descartados(est_emendadas_aut_temp, 'ANO')
print(f'{len(completos_emendadas_aut)}, {len(incompletos_emendadas_aut)}')

7, 10


In [28]:
df_est_bsb_conv_temp = df_temperatura_media_mes(est_bsb_conv_temp, ['ANO','NUM_MÊS','MÊS'], 'TEMPERATURA MEDIA COMPENSADA, DIARIA(°C)')
df_est_bsb_aut_temp = df_temperatura_media_mes(est_bsb_aut_temp, ['ANO','NUM_MÊS','MÊS'], 'TEMPERATURA MEDIA, DIARIA (AUT)(°C)')
df_est_emendadas_aut_temp = df_temperatura_media_mes(est_emendadas_aut_temp, ['ANO','NUM_MÊS','MÊS'], 'TEMPERATURA MEDIA, DIARIA (AUT)(°C)')

In [29]:
df_est_emendadas_aut_temp

,ANO,NUM_MÊS,MÊS,"TEMPERATURA MEDIA, DIARIA (AUT)(°C)"
0,2008,10,October,23.605167
1,2008,11,November,21.650417
2,2008,12,December,21.250538
3,2009,1,January,21.746324
4,2009,2,February,21.764020
...,...,...,...,...
181,2023,11,November,24.425556
182,2023,12,December,23.412769
183,2024,1,January,22.408192
184,2024,2,February,22.472797


In [31]:
# Aplicar filtro, ie, lista com os anos que possuem mais de 360 dias e atribuir a variável precip_mes_serie
est_bsb_conv_temp_serie  = est_bsb_conv_temp[est_bsb_conv_temp["ANO"].isin(completos_bsb_conv)]
est_bsb_aut_temp_serie = est_bsb_aut_temp[est_bsb_aut_temp["ANO"].isin(completos_bsb_aut)]
est_emendadas_aut_temp_serie = est_emendadas_aut_temp[est_emendadas_aut_temp["ANO"].isin(completos_emendadas_aut)]

In [32]:
df_est_bsb_conv_temp_mes = df_temperatura_media_ano_completo(est_bsb_conv_temp_serie, ['NUM_MÊS','MÊS'], 'TEMPERATURA MEDIA COMPENSADA, DIARIA(°C)')
df_est_bsb_aut_temp_mes = df_temperatura_media_ano_completo(est_bsb_aut_temp_serie, ['NUM_MÊS','MÊS'], 'TEMPERATURA MEDIA, DIARIA (AUT)(°C)')
df_est_emendadas_aut_temp_mes = df_temperatura_media_ano_completo(est_emendadas_aut_temp_serie, ['NUM_MÊS','MÊS'], 'TEMPERATURA MEDIA, DIARIA (AUT)(°C)')

In [33]:
df_est_bsb_conv_temp_mes

,NUM_MÊS,MÊS,"TEMPERATURA MEDIA COMPENSADA, DIARIA(°C)"
0,1,January,21.592577
1,2,February,21.630224
2,3,March,21.654972
3,4,April,21.247420
4,5,May,20.016548
5,6,June,18.997148
6,7,July,18.889711
7,8,August,20.707746
8,9,September,22.367396
9,10,October,22.438981


In [34]:
df_est_bsb_aut_temp_mes

,NUM_MÊS,MÊS,"TEMPERATURA MEDIA, DIARIA (AUT)(°C)"
0,1,January,21.820968
1,2,February,21.859253
2,3,March,21.749434
3,4,April,21.545502
4,5,May,20.177070
5,6,June,19.536150
6,7,July,19.346516
7,8,August,21.253149
8,9,September,23.585131
9,10,October,23.493068


In [35]:
df_est_emendadas_aut_temp_mes

,NUM_MÊS,MÊS,"TEMPERATURA MEDIA, DIARIA (AUT)(°C)"
0,1,January,21.774280
1,2,February,21.834511
2,3,March,21.803300
3,4,April,21.327742
4,5,May,20.176067
5,6,June,19.063689
6,7,July,18.931511
7,8,August,20.557528
8,9,September,22.874598
9,10,October,22.886051


In [36]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_est_bsb_conv_temp_mes['MÊS'], 
                         y=round(df_est_bsb_conv_temp_mes['TEMPERATURA MEDIA COMPENSADA, DIARIA(°C)'],2), 
                         mode='lines+markers', 
                         name='Estação Convencional Inmet (1961 - 2023)'))
fig.add_trace(go.Scatter(x=df_est_bsb_aut_temp_mes['MÊS'],
                         y=round(df_est_bsb_aut_temp_mes['TEMPERATURA MEDIA, DIARIA (AUT)(°C)'],2), 
                         mode='lines+markers', 
                         name= 'Estação Automática Inmet (2000 - 2023)'))
fig.add_trace(go.Scatter(x=df_est_emendadas_aut_temp_mes['MÊS'], 
                         y=round(df_est_emendadas_aut_temp_mes['TEMPERATURA MEDIA, DIARIA (AUT)(°C)'],2), 
                         mode='lines+markers', 
                         name='Estação Automática Águas Emendadas (2008 - 2023)'))


fig.update_layout(title='Média da Temperatura Média Mensal - Estação Convencional Inmet (1961 - 2023)', xaxis_title='Mês', yaxis_title='Temperatura Média (°C)')

In [37]:
print("Média da temperatura média - Est. Convencional Inmet: {}\n"
            "Média da temperatura média - Est. Automática Inmet: {}\n"
            "Média da temperatura média - Est. Automática Águas Emendadas: {}".format(
        df_est_bsb_conv_temp_mes['TEMPERATURA MEDIA COMPENSADA, DIARIA(°C)'].mean(),
        df_est_bsb_aut_temp_mes['TEMPERATURA MEDIA, DIARIA (AUT)(°C)'].mean(),
        df_est_emendadas_aut_temp_mes['TEMPERATURA MEDIA, DIARIA (AUT)(°C)'].mean()
))                                                                                                                                                                                                                                                                                           


Média da temperatura média - Est. Convencional Inmet: 21.038784588452803
Média da temperatura média - Est. Automática Inmet: 21.50174264454749
Média da temperatura média - Est. Automática Águas Emendadas: 21.200825137123797
